# Twitch PostgreSQL database

### PostgreSQL connection

Based on https://naysan.ca/2020/05/31/postgresql-to-pandas/

In [165]:
# Librairies
import psycopg2
import pandas as pd
import numpy as np
import sys
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/parthpahwa/nltk_data...


True

In [3]:
# Connection parameters to login
co_param = {
    "host"      : "twitch.caampywfg0rz.us-east-1.rds.amazonaws.com",
    "database"  : "Twitch",
    "user"      : "GaTech_team_96",
    "password"  : "i-love-my-coffee-without-milk-and-sugar-at-800AM"
}

In [4]:
def connect(co_param):
    """
    Connect to the PostgreSQL database server
    """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**co_param)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [5]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

### stream_data database

In [6]:
# SQL query
sql_query = """SELECT * FROM stream_data"""

# Column names
stream_data_col_names = ["game_id","stream_id","language","started_at","title",
                            "stream_type","user_id","user_name","viewer_count","user_login","game_name",
                            "thumbnail_url","tag_ids","is_mature","time_logged"]

# Retrieving the data
stream_data = postgresql_to_dataframe(connect(co_param), sql_query, stream_data_col_names)
# stream_data.head()

Connecting to the PostgreSQL database...
Connection successful


###

In [7]:
# changing formatting from 
stream_data.loc[:,"started_at"] = stream_data.loc[:,"started_at"].map(lambda x: x.rstrip("Z"))
stream_data.loc[:,"started_at"] = stream_data.loc[:,"started_at"].map(lambda x: x.replace("T", " "))
stream_data["stream_duration_hours"] = pd.to_datetime(stream_data["time_logged"])-pd.to_datetime(stream_data["started_at"])
stream_data['stream_duration_hours'] = stream_data['stream_duration_hours']/np.timedelta64(1, 'h')


# Changing is_mature with True =1 & False = 0
stream_data.loc[stream_data["is_mature"] == True, "is_mature"] = 1
stream_data.loc[stream_data["is_mature"] == False, "is_mature"] = 0

stream_data.head()

,game_id,stream_id,language,started_at,title,stream_type,user_id,user_name,viewer_count,user_login,game_name,thumbnail_url,tag_ids,is_mature,time_logged,stream_duration_hours
0,497057,40052420184,en,2022-11-09 11:02:36,Didn't Get Ghost Writer KEKW | OW2 Later!,live,96828357,Gumpita,60,gumpita,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},0,2022-11-09 12:25:26,1.380556
1,497057,40052300424,en,2022-11-09 10:03:38,"6 GMs, 3 Guardians, 1 stream. LET'S GO. // !ads",live,54943031,RoboticAdi,44,roboticadi,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},1,2022-11-09 12:25:26,2.363333
2,497057,46278752252,en,2022-11-09 11:24:23,GMs GMs GMs Help !wraith !merch !secretlab,live,126780483,GrandadGaming_,43,grandadgaming_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},1,2022-11-09 12:25:26,1.017500
3,497057,41479595547,en,2022-11-09 07:49:46,PvE n chill !throne !adorabullets,live,584365733,TalkQwertyToMe_,43,talkqwertytome_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},1,2022-11-09 12:25:26,4.594444
4,497057,40052489192,de,2022-11-09 11:33:33,Solo Flawless Großmeister Sog der Habsucht | !...,live,98447720,iPhil_,38,iphil_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{9166ad14-41f1-4b04-a3b8-c8eb838c6be6},1,2022-11-09 12:25:26,0.864722


0. filtering at english game
1. temp => get the diff of time_logged and started_at
2. how is the game behaving after 1week or 2? >>> defining a target for the MLalgo
        input target
        avg/medium/total viewers >>> 
3. Post-temp >>> sentiment analysis >>> title
4. Audience target > is_mature of the game
5. python dict for the categories of the game



### stream_data_ENG database

In [8]:
# Generating a temp database
df_temp = stream_data.copy()

# Selecting only English stream
df_temp = df_temp[df_temp["language"] == "en"]

# Computing the final table
stream_data_ENG = df_temp
stream_data_ENG.head()

,game_id,stream_id,language,started_at,title,stream_type,user_id,user_name,viewer_count,user_login,game_name,thumbnail_url,tag_ids,is_mature,time_logged,stream_duration_hours
0,497057,40052420184,en,2022-11-09 11:02:36,Didn't Get Ghost Writer KEKW | OW2 Later!,live,96828357,Gumpita,60,gumpita,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},0,2022-11-09 12:25:26,1.380556
1,497057,40052300424,en,2022-11-09 10:03:38,"6 GMs, 3 Guardians, 1 stream. LET'S GO. // !ads",live,54943031,RoboticAdi,44,roboticadi,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},1,2022-11-09 12:25:26,2.363333
2,497057,46278752252,en,2022-11-09 11:24:23,GMs GMs GMs Help !wraith !merch !secretlab,live,126780483,GrandadGaming_,43,grandadgaming_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},1,2022-11-09 12:25:26,1.017500
3,497057,41479595547,en,2022-11-09 07:49:46,PvE n chill !throne !adorabullets,live,584365733,TalkQwertyToMe_,43,talkqwertytome_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},1,2022-11-09 12:25:26,4.594444
5,497057,40466199561,en,2022-11-09 02:56:26,Helping the Community!,live,736149073,DankDoctor,31,dankdoctor,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},1,2022-11-09 12:25:26,9.483333


### df_game_avg_maturity database

In [9]:
# Generating a temp database
df = stream_data_ENG[["game_id", "time_logged", "is_mature"]].copy()

# changing the format of the col
df["game_id"] = df["game_id"].astype('str')
df["avg_maturity"] = df["is_mature"].astype('float')

# Grouping by "game_id" and "time_logged" and its avg "is_mature"
df_game_avg_maturity = df.groupby(["game_id", "time_logged"]).mean()
df_game_avg_maturity.head(10)

avg_maturity
game_id   time_logged                      
102007682 2022-11-09 12:24:55      0.483871
          2022-11-09 12:24:57      0.483871
          2022-11-09 12:25:20      0.483871
          2022-11-09 12:25:26      0.483871
          2022-11-09 12:25:45      0.500000
          2022-11-09 12:25:46      0.500000
          2022-11-09 12:25:47      0.500000
          2022-11-09 12:26:08      0.517241
          2022-11-09 12:26:10      0.517241
          2022-11-09 12:26:44      0.517241

### Twitch Tags and their categories

In [15]:
# Windows
# twitch_tags_cat = pd.read_csv('api_connection\Twitch_tags.csv')

# Linux/Mac
twitch_tags_cat = pd.read_csv('./api_connection/Twitch_tags.csv')

twitch_tags_cat.head()

,Category_or_Stream,TagName,TagId
0,Category,4X,7304b834-d065-47d5-9865-c19cd17d2639
1,Category,Action,4d1eaa36-f750-4862-b7e9-d0a13970d535
2,Category,Adventure Game,80427d95-bb46-42d3-bf4d-408e9bdca49a
3,Category,Arcade,7ff66192-68ef-4b69-8906-24736bf66ed0
4,Category,Autobattler,cd2ee226-342b-4e6b-90d5-c14687006b04


In [73]:
# twitch_tags_cat[['TagId', 'TagName']].to_dict()
twitch_tags_cat[twitch_tags_cat['TagId'] == '6ea6bca4-4712-4ab9-a906-e3336a9d8039']

,Category_or_Stream,TagName,TagId


### Encoding time of day into 6 blocks

In [184]:
MIN_STREAM_TIME_THRESHOLD = 0.5 # Hours
MAX_STREAM_TIME_THRESHOLD = 7 # Hours

df_with_encoded_time = stream_data_ENG.copy()
df_with_encoded_time['log_date'] = pd.to_datetime(df_with_encoded_time['time_logged']).dt.date
df_with_encoded_time['time_logged_encoded'] = pd.to_datetime(df_with_encoded_time['time_logged']).dt.hour
df_with_encoded_time['time_logged_encoded'] =  df_with_encoded_time['time_logged_encoded']//4

df_with_encoded_time = df_with_encoded_time[
    (df_with_encoded_time['stream_duration_hours'] > MIN_STREAM_TIME_THRESHOLD) & 
    (df_with_encoded_time['stream_duration_hours'] < MAX_STREAM_TIME_THRESHOLD)
].reset_index(drop=True)

In [186]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

si = SentimentIntensityAnalyzer()

def get_sentiment_row(row):
    sentiment = si.polarity_scores(row)
    return pd.Series([sentiment['pos'], sentiment['neg'], sentiment['neu']])

df_with_encoded_time['positive_sentiment'] = 0.0
df_with_encoded_time['negative_sentiment'] = 0.0
df_with_encoded_time['neutral_sentiment'] = 0.0

df_with_encoded_time[['positive_sentiment',
                     'negative_sentiment',
                     'neutral_sentiment']] =  df_with_encoded_time.title.apply(lambda row: get_sentiment_row(row))

### Feature Engineering

In [189]:
# Add mean, median, total viewership 
df_with_features = df_with_encoded_time.groupby(['time_logged_encoded', 'game_name', 'log_date'])[['viewer_count']].mean().reset_index()
df_with_features = df_with_features.rename(columns={'viewer_count': 'mean_viewer_count'})

temp_df = df_with_encoded_time.groupby(['time_logged_encoded', 'game_name', 'log_date'])[['viewer_count']].median().reset_index()
temp_df = temp_df.rename(columns={'viewer_count': 'median_viewer_count'})
df_with_features = pd.merge(df_with_features, temp_df)

temp_df = df_with_encoded_time.groupby(['time_logged_encoded', 'game_name', 'log_date'])[['viewer_count']].sum().reset_index()
temp_df = temp_df.rename(columns={'viewer_count': 'total_viewer_count'})
df_with_features = pd.merge(df_with_features, temp_df)


# Add mean, median, total stream time 
temp_df = df_with_encoded_time.groupby(['time_logged_encoded', 'game_name', 'log_date'])[['stream_duration_hours']].mean().reset_index()
temp_df = temp_df.rename(columns={'stream_duration_hours': 'mean_stream_duration_hours'})
df_with_features = pd.merge(df_with_features, temp_df)

temp_df = df_with_encoded_time.groupby(['time_logged_encoded', 'game_name', 'log_date'])[['stream_duration_hours']].median().reset_index()
temp_df = temp_df.rename(columns={'stream_duration_hours': 'median_stream_duration_hours'})
df_with_features = pd.merge(df_with_features, temp_df)

temp_df = df_with_encoded_time.groupby(['time_logged_encoded', 'game_name', 'log_date'])[['stream_duration_hours']].sum().reset_index()
temp_df = temp_df.rename(columns={'stream_duration_hours': 'total_stream_duration_hours'})
df_with_features = pd.merge(df_with_features, temp_df)


# Add average matrure rating 
temp_df = df_with_encoded_time.groupby(['time_logged_encoded', 'game_name', 'log_date'])[['is_mature']].mean().reset_index()
df_with_features = pd.merge(df_with_features, temp_df)


# Add mean sentiment
temp_df = df_with_encoded_time.groupby(['time_logged_encoded', 'game_name', 'log_date'])[['positive_sentiment']].mean().reset_index()
temp_df = temp_df.rename(columns={'positive_sentiment': 'mean_positive_sentiment'})
df_with_features = pd.merge(df_with_features, temp_df)

temp_df = df_with_encoded_time.groupby(['time_logged_encoded', 'game_name', 'log_date'])[['negative_sentiment']].mean().reset_index()
temp_df = temp_df.rename(columns={'negative_sentiment': 'mean_negative_sentiment'})
df_with_features = pd.merge(df_with_features, temp_df)

temp_df = df_with_encoded_time.groupby(['time_logged_encoded', 'game_name', 'log_date'])[['neutral_sentiment']].mean().reset_index()
temp_df = temp_df.rename(columns={'neutral_sentiment': 'mean_neutral_sentiment'})
df_with_features = pd.merge(df_with_features, temp_df)

df_with_features.head()

,time_logged_encoded,game_name,log_date,mean_viewer_count,median_viewer_count,total_viewer_count,mean_stream_duration_hours,median_stream_duration_hours,total_stream_duration_hours,is_mature,mean_positive_sentiment,mean_negative_sentiment,mean_neutral_sentiment
0,0,7 Days to Die,2022-11-10,14.445455,6.0,1589,2.206146,1.752222,242.676111,0.672727,0.108018,0.129764,0.762255
1,0,7 Days to Die,2022-11-11,8.787879,5.0,870,2.486546,2.053611,246.168056,0.717172,0.106212,0.138778,0.755020
2,0,A Little To The Left,2022-11-11,17.730769,8.0,461,1.598707,1.491667,41.566389,0.615385,0.111231,0.041654,0.847115
3,0,A Plague Tale: Requiem,2022-11-10,6.342857,3.0,222,2.068548,1.953056,72.399167,0.542857,0.099029,0.100400,0.800571
4,0,ASMR,2022-11-10,166.462810,80.0,20142,2.432158,1.991389,294.291111,0.404959,0.104025,0.039083,0.856893


In [190]:
feauture_column_names = list(df_with_features.columns)[3:-3]
feauture_column_names

['mean_viewer_count',
 'median_viewer_count',
 'total_viewer_count',
 'mean_stream_duration_hours',
 'median_stream_duration_hours',
 'total_stream_duration_hours',
 'is_mature']

In [191]:
shift_duration_days = 1

training_data_frame_dictionary = {
    
}
for time_slot in df_with_features.time_logged_encoded.unique():
    df_filtered_on_timeslot = df_with_features[df_with_features['time_logged_encoded'] == time_slot].copy()
    
    training_data_frame_dictionary[time_slot] = {
        
    }
    
    for game in df_filtered_on_timeslot.game_name.unique():
        df_filtered_on_game = df_filtered_on_timeslot[df_filtered_on_timeslot['game_name'] == game].copy()
        df_filtered_on_game = df_filtered_on_game.sort_values(by='log_date').reset_index(drop=True)
        
        
        for col in feauture_column_names:
            df_filtered_on_game['target_'+ col + '_' + str(shift_duration_days)] = df_filtered_on_game[col].shift(-shift_duration_days)
        
        training_data_frame_dictionary[time_slot][game] = df_filtered_on_game

In [192]:
"""
Dictionary with following hierarcy

-time slot
    - game name
        - DataFrame with targets 

""" 
training_data_frame_dictionary[0]['7 Days to Die']

,time_logged_encoded,game_name,log_date,mean_viewer_count,median_viewer_count,total_viewer_count,mean_stream_duration_hours,median_stream_duration_hours,total_stream_duration_hours,is_mature,mean_positive_sentiment,mean_negative_sentiment,mean_neutral_sentiment,target_mean_viewer_count_1,target_median_viewer_count_1,target_total_viewer_count_1,target_mean_stream_duration_hours_1,target_median_stream_duration_hours_1,target_total_stream_duration_hours_1,target_is_mature_1
0,0,7 Days to Die,2022-11-10,14.445455,6.0,1589,2.206146,1.752222,242.676111,0.672727,0.108018,0.129764,0.762255,8.787879,5.0,870.0,2.486546,2.053611,246.168056,0.717172
1,0,7 Days to Die,2022-11-11,8.787879,5.0,870,2.486546,2.053611,246.168056,0.717172,0.106212,0.138778,0.755020,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### game_info database

In [9]:
# SQL query
sql_query = """SELECT * FROM game_info"""

# Column names
game_info_col_names = ["game_id","game_name","game_picture_url","time_logged"]

# Retrieving the data
game_info = postgresql_to_dataframe(connect(co_param), sql_query, game_info_col_names)
game_info.head()

Connecting to the PostgreSQL database...
Connection successful


,game_id,game_name,game_picture_url,time_logged
0,743,Chess,https://static-cdn.jtvnw.net/ttv-boxart/743-{w...,2022-11-09 12:24:36
1,2748,Magic: The Gathering,https://static-cdn.jtvnw.net/ttv-boxart/2748-{...,2022-11-09 12:24:36
2,11989,StarCraft,https://static-cdn.jtvnw.net/ttv-boxart/11989_...,2022-11-09 12:24:36
3,12839,Heroes of Might and Magic III: The Restoration...,https://static-cdn.jtvnw.net/ttv-boxart/12839_...,2022-11-09 12:24:36
4,12924,Warcraft III,https://static-cdn.jtvnw.net/ttv-boxart/12924-...,2022-11-09 12:24:36


### Random Code

In [124]:
# stream_data_ENG
# ast.literal_eval(stream_data_ENG.tag_ids[0])
import re
h = re.compile("['\{][0-9A-Za-z\-]+[,\}]")

h.match(stream_data_ENG.tag_ids[50])[0][1:-1]

'6ea6bca4-4712-4ab9-a906-e3336a9d8039'